# Sliced solid torus

In this notebook, we show how easy it is to create more complicated structures using set operations with separators, by slicing a torus.

In [1]:
using IntervalConstraintProgramming, ValidatedNumerics

In [2]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

half_plane = @constraint (x + y) + z <= 1

forward = IntervalConstraintProgramming.GeneratedFunction(Symbol[:x,:y,:z],Symbol[:_z_1_,:_z_2_,:_z_3_,:_z_4_,:_z_5_,:_z_6_,:_z_7_,:_z_8_],quote 
    _z_1_ = x ^ 2
    _z_2_ = y ^ 2
    _z_3_ = _z_1_ + _z_2_
    _z_4_ = sqrt(_z_3_)
    _z_5_ = 3 - _z_4_
    _z_6_ = _z_5_ ^ 2
    _z_7_ = z ^ 2
    _z_8_ = _z_6_ + _z_7_
end)
backward = IntervalConstraintProgramming.GeneratedFunction(Symbol[:x,:y,:z,:_z_1_,:_z_2_,:_z_3_,:_z_4_,:_z_5_,:_z_6_,:_z_7_,:_z_8_],Symbol[:x,:y,:z],quote 
    (_z_8_,_z_6_,_z_7_) = plus_rev(_z_8_,_z_6_,_z_7_)
    (_z_7_,z,_) = power_rev(_z_7_,z,2)
    (_z_6_,_z_5_,_) = power_rev(_z_6_,_z_5_,2)
    (_z_5_,_,_z_4_) = minus_rev(_z_5_,3,_z_4_)
    (_z_4_,_z_3_) = sqrt_rev(_z_4_,_z_3_)
    (_z_3_,_z_1_,_z_2_) = plus_rev(_z_3_,_z_1_,_z_2_)
    (_z_2_,y,_) = power_rev(_z_2_,y,2)
    (_z_1_,x,_) = power_rev(_z_1_,x,2)
end)
code = quote  # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 144:
    (_A_,x,y,z)->begin  # /Users/dpsanders/.julia

Separator:
- variables: x, y, z
- expression: (x + y) + z ∈ [-∞, 1]


In [3]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = pave(solid_torus ∩ half_plane, Y, 0.25);

  3.328242 seconds (7.76 M allocations: 313.713 MB, 3.67% gc time)


In [21]:
@time refine!(paving, 0.1)

 12.920378 seconds (43.82 M allocations: 1.598 GB, 5.94% gc time)


0.1

In [20]:
length(paving.inner)

68977

## 3D visualization

In [4]:
inner = paving.inner
boundary = paving.boundary;

In [5]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

Set up some colors that depend on the position:

In [6]:
positions = Point{3, Float32}[Point3(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors1 = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.1f0)
        for i in 1:length(inner)]
colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
colors = vcat(colors1, colors2);

In [7]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)


@async renderloop(window)

Task (runnable) @0x0000000110be9450